# quake 3 server connector



## discord api




### channel message poller



#### the code

poll discord channel?

discord api?



In [4]:
var fs = require('fs')
var path = require('path')
var {request} = require('gaxios')
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE
var credentials
var tokenPath
if(fs.existsSync('./discord-bot.txt')) {
    tokenPath = path.resolve('./discord-bot.txt')
} else {
    tokenPath = path.join(PROFILE_PATH, '.credentials/discord-bot.txt')
}
var token = fs.readFileSync(tokenPath).toString('utf-8').trim()

var DEFAULT_GUILD = process.env.DEFAULT_GUILD || '393252386426191873'
var DEFAULT_CHANNEL = process.env.DEFAULT_CHANNEL || '393252386426191875'
var DEFAULT_API = process.env.DEFAULT_API || 'https://discord.com/api/v6/'
var MESSAGE_TIME = process.env.DEFAULT_TIME || 1000 * 60 * 60 // 1 hour to respond
var DEFAULT_RATE = 1100

var delay = async () => await new Promise(resolve => setTimeout(resolve, DEFAULT_RATE))

async function userGuilds(userId = '@me') {
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`
        },
        method: 'GET',
        url: `${DEFAULT_API}users/${userId}/guilds`
    })
    await delay()
    return result.data
}

async function userChannels(userId = '@me') {
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`
        },
        method: 'GET',
        url: `${DEFAULT_API}channels/${userId}`
    })
    await delay()
    return result.data
}

async function userConnections(userId = '@me') {
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`
        },
        method: 'GET',
        url: `${DEFAULT_API}users/${userId}/connections`
    })
    await delay()
    return result.data
}

async function guildChannels(guildId = DEFAULT_GUILD) {
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`
        },
        method: 'GET',
        url: `${DEFAULT_API}guilds/${guildId}/channels`
    })
    await delay()
    return result.data
}

async function channelMessages(channelId = DEFAULT_CHANNEL) {
    var params = {
        after: (BigInt(Date.now() - 1420070400000 - MESSAGE_TIME) << BigInt(22)).toString()
    };
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`
        },
        method: 'GET',
        url: `${DEFAULT_API}channels/${channelId}/messages`,
        params
    })
    await delay()
    return result.data
}

async function triggerTyping(channelId = DEFAULT_CHANNEL) {
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`
        },
        method: 'POST',
        url: `${DEFAULT_API}channels/${channelId}/typing`
    })
    await delay()
    return result.data
}

async function createMessage(message, channelId = DEFAULT_CHANNEL) {
    var params = {
        content: message
    }
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`,
            'Content-Type': 'application/json'
        },
        method: 'POST',
        url: `${DEFAULT_API}channels/${channelId}/messages`,
        data: JSON.stringify(params)
    })
    await delay()
    return result.data
}

module.exports = {
    userGuilds,
    userChannels,
    userConnections,
    guildChannels,
    channelMessages,
    triggerTyping,
    createMessage
}


{
  userGuilds: [AsyncFunction: userGuilds],
  guildChannels: [AsyncFunction: guildChannels],
  channelMessages: [AsyncFunction: channelMessages],
  triggerTyping: [AsyncFunction: triggerTyping],
  createMessage: [AsyncFunction: createMessage]
}

### authorize discord oauth



#### the code

authorize discord?


In [ ]:
var fs = require('fs')
var path = require('path')
var WebSocket = require('ws')
var {request} = require('gaxios')
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE
var tokenPath
if(fs.existsSync('./discord-bot.txt')) {
    tokenPath = path.resolve('./discord-bot.txt')
} else {
    tokenPath = path.join(PROFILE_PATH, '.credentials/discord-bot.txt')
}
var token = fs.readFileSync(tokenPath).toString('utf-8').trim()

var DEFAULT_API = process.env.DEFAULT_API || 'https://discord.com/api/v6/'

var heartbeat
var ws = false
var cancelConnection
var seq = 0
var privateChannels = {}

function sendHeartbeat() {
    if(!ws) return
    console.log('Sending heartbeat')
    ws.send(JSON.stringify({
        op: 1,
        d: seq
    }))
    cancelConnection = setTimeout(() => ws ? ws.close() : false, 4000)
}

async function authorizeGateway() {
    if(ws) return
    var result
    try {
        result = await request({
            headers: {
                Authorization: `Bot ${token}`
            },
            method: 'GET',
            url: `${DEFAULT_API}gateway/bot`
        })
    } catch (e) {
        console.log(e.message)
        return
    }
    ws = new WebSocket(result.data.url)
    ws.on('open', () => {
        console.log('Connecting to Discord')
    })
    var identified = false
    ws.on('message', (message) => {
        var msgBuff = new Buffer.from(message)
        var gateway = JSON.parse(msgBuff.toString('utf-8'))
        if(gateway.s) seq = gateway.s
        if(gateway.d && gateway.d.seq) seq = gateway.d.seq
        if(gateway.op == 10) {
            heartbeat = setInterval(sendHeartbeat, gateway.d.heartbeat_interval)
            ws.send(JSON.stringify({
                op: 2,
                intents: [
                    'DIRECT_MESSAGES',
                    'GUILD_MESSAGES',
                    'GUILDS'
                ], //1 | 1 << 9 | 1 << 10 || 1 << 11 | 1 << 12 | 1 << 13 | 1 << 14
                d: {
                    token: token,
                    properties: {
                        "$os": "linux",
                        "$browser": "jupyter",
                        "$device": "quake3"
                    }
                }
            }))
            return
        } else if (gateway.op === 0 || gateway.op === 9) {
            identified = true
            if(gateway.t == 'MESSAGE_CREATE') {
                privateChannels[gateway.d.channel_id] = Date.now()
            }
            return
        } else if (gateway.op === 11) {
            clearTimeout(cancelConnection)
            return
        }
        console.log(gateway)
    })
    var timer
    ws.on('close', () => {
        console.log('Discord disconnected')
        if(timer) clearInterval(timer)
        clearInterval(heartbeat)
        ws.close()
        ws = false
    })
    await new Promise(resolve => {
        timer = setInterval(() => {
            if(identified) {
                clearInterval(timer)
                resolve()
            }
        }, 1000)
    });
}

/*
async function authorizeDiscord() {
    var {client_id, client_secret} = JSON.parse(fs.readFileSync(credentials).toString('utf-8'))
    if(fs.existsSync(tokenPath))
        var {access_token, expires} = JSON.parse(fs.readFileSync(tokenPath).toString('utf-8'))
    if(access_token && Date.now() < expires) return access_token
    var data = `grant_type=client_credentials&scope=identify%20connections%20bot%20guilds%20messages.read%20rpc`
    var result = await request({
        headers: {
            'Content-Type': 'application/x-www-form-urlencoded',
            Authorization: 'Basic ' + Buffer.from(`${client_id}:${client_secret}`).toString('base64')
        },
        method: 'POST',
        url: `https://discord.com/api/v6/oauth2/token`,
        data
    })
    fs.writeFileSync(tokenPath, JSON.stringify(Object.assign({}, result.data, {
        expires: (result.data.expires_in * 1000) + Date.now()}), null, 2))
    return result.data.access_token
}
*/

module.exports = {
    authorizeGateway,
    privateChannels
}


### respond discord commands?



#### the code



In [ ]:
var importer = require('../Core')
var discordApi = importer.import('discord api')
var {authorizeGateway, privateChannels} = importer.import('authorize discord')

async function readAllCommands(specificChannel) {
    // matching format  @megamind  challenge freon dm17 , :thumbsup:   :thumbsdown: .
    var private = false
    var messages = []
    var responses = []
    var channels = []
    var commands = []
    var launches = []
    
    if(specificChannel == '@me') {
        // only read channel if it was updated within the last hour
        var userChannels = Object
            .keys(privateChannels)
            .filter(k => privateChannels[k] > Date.now() - 1000 * 60 * 60)
            .map(k => ({id: k}))
        channels.push.apply(channels, userChannels)
        specificChannel = ''
        private = true
    } else {
        var guilds = await discordApi.userGuilds()
        console.log(`Reading ${guilds.length} guilds`)
        for(var i = 0; i < guilds.length; i++) {
            channels.push.apply(channels, await discordApi.guildChannels(guilds[i].id))
        }
    }
    
    console.log(`Reading ${channels.length} channels`)
    for(var i = 0; i < channels.length; i++) {
        if(!specificChannel
           || channels[i].id == specificChannel
           || (typeof specificChannel == 'string'
              && (specificChannel.length === 0
                 || (channels[i].name
                     && channels[i].name.match(new RegExp(specificChannel, 'ig'))
                    )
                 )
              )
          )
            messages.push.apply(messages, await discordApi.channelMessages(channels[i].id))
    }
    
    // find commands in channel history
    console.log(`Reading ${messages.length} messages`)
    for(var j = 0; j < messages.length; j++) {
        if(Object.keys(discordCommands)
           .filter(k => messages[j].content.match(discordCommands[k])).length > 0) {
            messages[j].private = private
            commands.push(messages[j])
            if((messages[j].reactions || [])
                .filter(a => a.emoji.name == '\u{1F44D}').length > 0) {
                launches.push(messages[j])
            }
        }
        if(messages[j].content.match(/```BOT/ig)) {
            responses.push(messages[j])
            if((messages[j].reactions || [])
                .filter(a => a.emoji.name == '\u{1F44D}').length > 0) {
                var l = messages.filter(m => messages[j].content.match('```BOT'+m.id))[0]
                if(!l) continue
                l.launching = true
                l.reactions = l.reactions || []
                l.reactions.push.apply(l.reactions, messages[j].reactions)
                if(l) launches.push(l)
            }
        }
    }
    
    // exclude commands that already got a response
    return commands
        .filter(c => responses.filter(r => r.content.match(new RegExp('```BOT'+c.id))).length === 0)
        .concat(launches)
        .filter(c => responses.filter(r => r.content.match(new RegExp('```BOT'+c.id+'L'))).length === 0)
        .filter((c, i, arr) => arr.indexOf(c) === i)
}

async function respondCommand(specificChannel) {
    await authorizeGateway()
    var commands = await readAllCommands(specificChannel)
    for(var i = 0; i < commands.length; i++) {
        var options = (discordCommands.CHALLENGE).exec(commands[i].content)
        if(options) challengeCommand.apply(null, [options[1], options[2]])
        //options = (discordCommands.CHALLENGE).exec(commands[i].content)
        //if(options) challengeCommand.apply(null, [options[1], options[2]])
    }
}

module.exports = respondCommand


#### test specific channel?



In [ ]:
var importer = require('../Core')
var respondCommand = importer.import('respond discord commands')

module.exports = function testChannel(channel) {
    respondCommand(channel)
    
}


### challenge discord command?



#### the code



In [ ]:
var importer = require('../Core')
var discordApi = importer.import('discord api')
var serverApi = importer.import('quake 3 server commands')

var DEFAULT_HOST = process.env.DEFAULT_HOST || 'http://quakeiiiarena.com/play/'
var MODS = typeof process.env.DEFAULT_MODS == 'string'
    ? JSON.parse(process.env.DEFAULT_MODS)
    : [
        'baseq3',
        'freon'
    ]

async function challengeCommand(launch, map) {
    var message = 'I read you'
    var instruction = ''
    if(!MODS.includes(launch) && map.length === 0) {
        map = launch
        launch = ''
    } else if(map.length === 0) {
        map = 'q3dm17'
    }
    if(launch.length == 0) {
        instruction += ', assuming baseq3 on map ' + map
    } else if(commands[i].launching) {
        instruction += ' ' + launch + ' on map ' + map
    }
    if(!commands[i].launching && !commands[i].content.match(/:thumbsup:/ig)) {
        message = 'Waiting for reaction'
        instruction += ', react with :thumbsup: to launch'
    }
    if(commands[i].launching) {
        message = 'Launching'
        await discordApi.createMessage(message + instruction + '```BOT'+commands[i].id+'L\nbeep boop\n```', commands[i].channel_id)
        await discordApi.triggerTyping(commands[i].channel_id)
        var masters = await serverApi.listMasters(void 0, void 0, false)
        if(masters.length === 0) {
            await discordApi.createMessage(`Boo hoo, no servers avaibale. :cry:` + '```BOT'+commands[i].id+'L\nbeep boop\n```', commands[i].channel_id)
            return
        }
        serverApi.sendRcon(masters[0].ip, masters[0].port, '\exec ' + launch + '.cfg')
        serverApi.sendRcon(masters[0].ip, masters[0].port, '\map ' + map)
        await new Promise(resolve => setTimeout(resolve, 1000))
        await discordApi.createMessage(`Match is ready ${DEFAULT_HOST}?connect%20${masters[0].ip}:${masters[0].port} (${masters[0].ip}:${masters[0].port})`
                                       + '```BOT'+commands[i].id+'L\nbeep boop\n```', commands[i].channel_id)
    } else if (instruction.length > 0) {
        await discordApi.createMessage(message + instruction + '```BOT'+commands[i].id+'\nbeep boop\n```', commands[i].channel_id)
    }
}

module.exports = challengeCommand


### direct messages and discord rcon



#### the code

discord bot?


In [ ]:
var importer = require('../Core')
var challengeCommand = importer.import('challenge discord command')

var personality = [
    'Yeehaw!',
    'Balls to wall!',
    'Do it to it!',
    'Got it!',
    'Let\'s play!',
    'Roger that!',
    'I read you!',
    'Buenos Dias!'
]

var lose = [
    'Error. Error.',
    'Oops.',
    'Boo hoo!',
    'Phooey!',
    'Au revoir, mon amis.',
    '#*&^@#!!',
]

var commands = {
    CHALLENGE: /(@\w\s*chall?[ae]nge|chall?[ae]nge\s*@\w)\s*([^:@\s]*?)\s*([^:@\s]*?)/ig,
    CONNECT: /conn?ect\s*([0-9\.a-z-_]*)/ig,
    RCON: /rcon(pass?wo?rd)?\s*(.+)/ig,
    DISCONNECT: /disconn?ect/ig,
    CONFIG: /\.cfg|config|configure/ig,
    LOAD: /load\s*(\w*)/ig,
}

module.exports = {
    challengeCommand,
}


## quake 3 commands



### quake 3 server commands



#### the code

quake 3 server commands?


In [ ]:
var path = require('path')
var fs = require('fs')
var zlib = require('zlib')
var dgram = require('dgram')
var udpClient = dgram.createSocket('udp4')
var importer = require('../Core')
var {
    getServersResponse, statusResponse, infoResponse
} = importer.import('quake 3 server responses')
udpClient.on('message', updateInfo)

var MAX_TIMEOUT = process.env.DEFAULT_TIMEOUT || 10000
var DEFAULT_MASTER = process.env.DEFAULT_MASTER || '192.168.0.4'
var DEFAULT_PASS = process.env.DEFAULT_PASS || 'password123!'

var masters = []
function mergeMaster(master) {
    var found = false
    masters.forEach((ma, i) => {
        if(ma['ip'] == master['ip'] && ma['port'] == master['port']) {
            found = true
            Object.assign(masters[i], master)
            return false
        }
    })
    if(!found)
        masters.push(master)
}

async function updateInfo(m, rinfo) {
    if(m[0] == 255 && m[1] == 255 && m[2] == 255 && m[3] == 255)
        m = m.slice(4, m.length)
    if(m.slice(0, 'getserversResponse'.length).toString('utf-8') == 'getserversResponse') {
        var mas = getServersResponse(m)
        mas.forEach(master => {
            mergeMaster(master)
            getStatus(master.ip, master.port)
        })
    } else if (m.slice(0, 'statusResponse'.length).toString('utf-8') == 'statusResponse') {
        var info = statusResponse(m)
        mergeMaster(Object.assign(info, {
            ip: rinfo.address,
            port: rinfo.port
        }))
    } else if (m.slice(0, 'infoResponse'.length).toString('utf-8') == 'infoResponse') {
        var info = infoResponse(m)
        mergeMaster(Object.assign(info, {
            ip: rinfo.address,
            port: rinfo.port
        }))
    }
}

function sendRcon(address, port, command, password = DEFAULT_PASS) {
    var msgBuff = new Buffer.from(`\xFF\xFF\xFF\xFFrcon "${password}" ${command} ;`.split('').map(c => c.charCodeAt(0)))
    udpClient.send(msgBuff, 0, msgBuff.length, port, address)
}

function getStatus(address, port) {
    var msgBuff = new Buffer.from('\xFF\xFF\xFF\xFFgetstatus'.split('').map(c => c.charCodeAt(0)))
    udpClient.send(msgBuff, 0, msgBuff.length, port, address)
}

function getInfo(address, port) {
    var msgBuff = new Buffer.from('\xFF\xFF\xFF\xFFgetinfo xxx'.split('').map(c => c.charCodeAt(0)))
    udpClient.send(msgBuff, 0, msgBuff.length, port, address)
}

async function listMasters(master = DEFAULT_MASTER, port = 27950, wait = true) {
    var msgBuff = new Buffer.from('\xFF\xFF\xFF\xFFgetservers 68 empty'.split('').map(c => c.charCodeAt(0)))
    udpClient.send(msgBuff, 0, msgBuff.length, port, master)
    if(wait) {
        await new Promise(resolve => setTimeout(resolve, MAX_TIMEOUT))
    } else {
        var timeout = 0
        var timer
        await new Promise(resolve => {
            timer = setInterval(() => {
                if(masters.filter(s => s.gamename).length > 0 || timeout == MAX_TIMEOUT / 20) {
                    resolve()
                } else {
                    timeout += 20
                }
            }, 20)
        })
        clearInterval(timer)
    }
    return masters
}

module.exports = {
    listMasters,
    getInfo,
    getStatus,
    sendRcon,
    udpClient
}


#### test quake 3 rcon commands?



In [ ]:
var importer = require('../Core')
var serverApi = importer.import('quake 3 server commands')

async function testRcon (command) {
    var masters = await serverApi.listMasters(void 0, void 0, false)
    serverApi.sendRcon(masters[0].ip, masters[0].port, command)
    await new Promise(resolve => setTimeout(resolve, 1000))
    serverApi.udpClient.close()
}

module.exports = testRcon


### quake 3 server responses



#### the code

quake 3 server responses?



In [ ]:

function getServersResponse(m) {
    var masters = []
    m = m.slice('getserversResponse'.length)
    for(var i = 0; i < m.length / 7; i++) {
        var ip = i*7+1
        if(m[ip-1] !== '\\'.charCodeAt(0)) continue
        if(m.slice(ip, ip+3) == 'EOT') continue
        var master = {
            ip: m[ip] + '.' + m[ip+1] + '.' + m[ip+2] + '.' + m[ip+3],
            port: (m[ip+4] << 8) + m[ip+5],
        }
        masters.push(master)
    }
    return masters
}

function statusResponse(m) {
    m = m.slice('statusResponse'.length)
    var config = m.toString('utf-8')
        .trim().split(/\n/ig)[0]
        .trim().split(/\\/ig).slice(1)
    return config.reduce((obj, c, i, arr) => {
        if(i & 1) obj[arr[i-1]] = c
        return obj
    }, {})
}

function infoResponse(m) {
    m = m.slice('infoResponse'.length)
    var config = m.toString('utf-8')
        .trim().split(/\n/ig)[0]
        .trim().split(/\\/ig).slice(1)
    return config.reduce((obj, c, i, arr) => {
        if(i & 1) obj[arr[i-1]] = c
        return obj
    }, {})
}

module.exports = {
    getServersResponse,
    statusResponse,
    infoResponse,
}


### Huffman decoder



#### the code

huffman decode?


In [ ]:
var Huffman = require('/Users/briancullinan/planet_quake/code/xquakejs/lib/huffman.js')
var MAX_STRING_CHARS = 1024
var buffer
Huffman.onRuntimeInitialized = () => {
    Huffman['_MSG_initHuffman']()
    buffer = Huffman.allocate(new Int8Array(MAX_STRING_CHARS), 'i8', 0)
}


function readBits(m, offset, bits = 8) {
    var value
    var nbits = bits & 7
    var sym = Huffman.allocate(new Int32Array(1), 'i32', 1)
    var bitIndex = offset
    m.forEach((c,i) => Huffman.HEAP8[buffer+i] = c)
    if ( nbits )
    {
        for ( i = 0; i < nbits; i++ ) {
            value |= Huffman._HuffmanGetBit( buffer, bitIndex ) << i
            bitIndex++
        }
        bits -= nbits
    }
    if ( bits )
    {
        for ( i = 0; i < bits; i += 8 )
        {
            bitIndex += Huffman._HuffmanGetSymbol( sym, buffer, bitIndex )
            value |= ( Huffman.getValue(sym) << (i+nbits) )
        }
    }
    return value
}


## entry files



### cron entry



#### the code

index?



In [ ]:
var importer = require('../Core')
var respondCommand = importer.import('respond discord commands')

var DEFAULT_CHANNEL = process.env.DEFAULT_CHANNEL || 'general'

var stillRunning = false
async function startResponder() {
    if(stillRunning) {
        console.log('Still running...')
        return
    }
    stillRunning = true
    try {
        await respondCommand(DEFAULT_CHANNEL)
        await respondCommand('@me')
    } catch (e) {
        console.log(e)
    }
    stillRunning = false
}
setInterval(startResponder, 5000)

module.exports = startResponder
